# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from time import sleep
#import re

# Work

In [31]:
base_url = 'http://diariooficial.rn.gov.br/dei/dorn3/'

binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

driver = webdriver.Firefox(options=options, capabilities=cap, executable_path='drivers\geckodriver.exe')
#driver = webdriver.Firefox(capabilities=cap, executable_path='drivers\geckodriver.exe')
driver.get(base_url)

In [32]:
materias = []

In [33]:
data = driver.find_element_by_class_name('ui-datepicker-days-cell-over.ui-datepicker-current-day')
ano = data.get_attribute('data-year')
mes = str(int(data.get_attribute('data-month')) + 1)
dia = data.find_element_by_tag_name('a').get_attribute('text')
data_formatada = (dia + '/0' + mes + '/' + ano if int(mes) < 10 else dia + '/' + mes + '/' + ano)
data_formatada = ('0' + data_formatada if int(dia) < 10 else data_formatada)
data_formatada

'19/10/2019'

In [12]:
def processar_linhas(link, text, blank):
    caminho = driver.find_element_by_id("lblSessaoPath").text
    lista_caminho = caminho.split("\\")[:-1]
    if blank != 0:
        [lista_caminho.append('') for b in range(blank)]
    if lista_caminho[0] == "DIÁRIO DA JUSTIÇA":
        lista_caminho.insert(0,'DIÁRIO OFICIAL')
    lista_caminho.insert(0, data_formatada)
    lista_caminho.append(text)
    lista_caminho.append(link)
    print(lista_caminho)
    return lista_caminho

In [13]:
def navegar(link, blank=5):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            materia = processar_linhas(link=subsessoes.text, text='Não Possui', blank=blank)
            materias.append(materia)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            materia = processar_linhas(link=link, text=f.text, blank=blank)
            materias.append(materia)
        try:
            tabela = driver.find_element_by_id("dgSubSessoes")
            links = tabela.find_elements_by_tag_name("a")
            return links
        except:
            return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

In [14]:
def crawl_diario(link_sessao):
    #sleep(1)
    sub1 = navegar(link_sessao)
    if len(sub1) != 0:
        id1 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub1)+2)]
        for i1 in id1:
            #sleep(1)
            #print(i1)
            s1 = driver.find_element_by_id(i1)
            sub2 = navegar(s1, blank=4)
            if len(sub2) == 0:
                #sleep(1)
                driver.back()
            else:
                id2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub2)+2)]
                for i2 in id2:
                    #sleep(1)
                    s2 = driver.find_element_by_id(i2)
                    sub3 = navegar(s2, blank=3)
                    if len(sub3) == 0:
                        #sleep(1)
                        driver.back()
                    else:
                        id3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub3)+2)]
                        for i3 in id3:
                            #sleep(1)
                            s3 = driver.find_element_by_id(i3)
                            sub4 = navegar(s3, blank=2)
                            if len(sub4) == 0:
                                #sleep(1)
                                driver.back()
                            else:
                                id4 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub4)+2)]
                                for i4 in id4:
                                    #sleep(1)
                                    s4 = driver.find_element_by_id(i4)
                                    sub5 = navegar(s4, blank=1)
                                    if len(sub5) == 0:
                                        #sleep(1)
                                        driver.back()
                                    else:
                                        id5 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub5)+2)]
                                        for i5 in id5:
                                            #sleep(1)
                                            s5 = driver.find_element_by_id(i5)
                                            sub6 = navegar(s5, blank=0)
                                            if len(sub6) == 0:
                                                #sleep(1)
                                                driver.back()
                                            if i5 == id5[-1]:
                                                #sleep(1)
                                                driver.back()
                                    if i4 == id4[-1]:
                                        #sleep(1)
                                        driver.back()
                            if i3 == id3[-1]:
                                #sleep(1)
                                driver.back()
                    if i2 == id2[-1]:
                        #sleep(1)
                        driver.back()
            if i1 == id1[-1]:
                #sleep(1)
                driver.back()

segunda versão

In [34]:
elements = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
#lista = navegar(sessoes[7])
#lista = []
for e in elements:
    s = driver.find_element_by_id(e)
    crawl_diario(s)
print(materias)
#lista

['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSULTORIA GERAL DO ESTADO (CGE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONTROLADORIA GERAL DO ESTADO (CONTROL)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'DEFENSORIA PÚBL

['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETÁRIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, DO ESPORTE E DO LAZER (SEEC)', 'FUNDAÇÃO JOSÉ AUGUSTO (FJA)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETÁRIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, DO ESPORTE E DO LAZER (SEEC)', 'INSTITUTO DE FORMAÇÃO DE PROFESSORES PRESIDENTE KENNEDY (IFESP)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETÁRIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, DO ESPORTE E DO LAZER (SEEC)', 'UNIVERSIDADE DO ESTADO DO RIO GRANDE DO NORTE (UERN)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA SAÚDE PÚBLICA (

['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / 6 URT', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / 7 URT', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / CRF', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / GABINETE', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETA

['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'DEFENSORIA PÚBLICA DO ESTADO (DPE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'GABINETE CIVIL DO GOVERNADOR DO ESTADO', 'CONSELHO ESTADUAL DOS DIREITOS DA CRIANÇA E DO ADOLESCENTE', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'GABINETE DO VICE-GOVERNADOR', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'LEIS', '', '', '', 'Lei nº 10.607 - UP - ONG Cidadão Esperança.doc', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191019&id_doc=662793']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'LEIS', '', '', '', 'Lei nº 10.608 - UP - Augusta e Respeitável Loja

['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD)', '', '', '', 'PORTARIA Nº 1337_2019 - GS_SEAD - folha NOVEMBRO.doc', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191019&id_doc=662848']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD)', 'COMPANHIA DE PROCESSAMENTO DE DADOS DO RN (DATANORTE) ', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD)', 'INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO - (IPERN) ', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO PENITENCIÁRIA (SEAP)', '', '', '', 'Não Possui', 'Nenhuma 

...

['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'UNIDADE CÍVEL DA FARN', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'UNIDADE CÍVEL DA MICROEMPRESA', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'UNIDADE CÍVEL DA UFRN', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'UNIDADE CÍVEL DA UNIVERSIDADE POTIGUAR - UNP', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2

['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '15ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '16ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '17ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '18ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPIT

['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '10- VARA DA JUSTICA FEDERAL-SUBSECAO DE MOSSORO DRA.CINTIA MENEZES BRUNETTA', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '11 VARA DA JUSTICA FEDERAL-SUBSECAO DE ASSU-DR.ALMIRO JOSE DA ROCHA LEMOS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '12 VARA-SUBSEÇAO DE PAU DOS FERROS -DR. JOSÉ CARLOS DANTAS T. DE SOUZA', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '1ª TURMA RECURSAL', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA',

['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS ADMINISTRATIVOS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DA PRESIDÊNCIA E CORREGEDORIA', 'ATOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DA PRESIDÊNCIA E CORREGEDORIA', 'AVISOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DA PRESIDÊNCIA E CORREGEDORIA', 'DESPACHOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DA PRESIDÊNCIA E CORREGEDORIA', 'EDI

['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'CONTRATOS, EDITAIS E AVISOS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'DECISÕES', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'DESPACHOS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'EDITAIS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'HABEAS CORPUS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['19/10/2019', 'DIÁRIO OFICIAL', 'DI

In [35]:
driver.quit()

In [36]:
df = pd.DataFrame(materias,columns=['data_publicacao', 'diario', 'secao', 'subsecao1', 'subsecao2', 'subsecao3', 'subsecao4', 'subsecao5', 'nome_materia', 'materia'])
df.head(3)

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
1,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
2,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...


In [37]:
df[df['nome_materia'] == 'Não Possui'].diario.count()

314

In [38]:
df.shape

(417, 10)

In [39]:
filename = "diário - {}".format(data_formatada.replace('/','_'))
df.to_csv(('datasets/diários/' + filename + '.csv'), index=False)

# Pegar texto das matérias

In [40]:
materias_df = df[df['nome_materia'] != 'Não Possui'].copy()
materias_df.head(3)

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
5,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",GABINETE CIVIL DO GOVERNADOR DO ESTADO,,,,GAC - Extrato Jurídico.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
9,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO DE LICITAÇÃO - TOMADA DE PREÇOS N° 015_2...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [41]:
materias_df.shape

(103, 10)

In [42]:
print(materias_df.materia.head(3))

5     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
9     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10    http://diariooficial.rn.gov.br/dei/dorn3/docvi...
Name: materia, dtype: object


In [43]:
def pegar_frame(link):
    r = requests.get(link)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    frame = soup.find_all('frame')[1]
    frame_link = frame.get('src')
    url_materia_oficial = base_url + frame_link
    return url_materia_oficial

In [44]:
materias_df['link_materia_oficial'] = materias_df['materia'].apply(pegar_frame)
materias_df.head(3)

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial
5,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",GABINETE CIVIL DO GOVERNADOR DO ESTADO,,,,GAC - Extrato Jurídico.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
9,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
10,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO DE LICITAÇÃO - TOMADA DE PREÇOS N° 015_2...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...


In [45]:
print(materias_df.link_materia_oficial.iloc[50])

http://diariooficial.rn.gov.br/dei/dorn3/documentos/00000001/20191019/662822.htm


In [46]:
def pegar_texto(link):
    global cont
    r = requests.get(link)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    #if len(soup.find_all('table')) > 0:
    #    soup('table')[0].decompose()
    conteudo_materia = soup.find_all('body')[0].text.strip()
    #conteudo_materia = re.sub("(<!--.*?-->)", "", noticia, flags=re.DOTALL)
    return conteudo_materia

In [47]:
materias_df['texto'] = materias_df['link_materia_oficial'].apply(pegar_texto)
materias_df.head(3)

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
5,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",GABINETE CIVIL DO GOVERNADOR DO ESTADO,,,,GAC - Extrato Jurídico.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,GABINETE CIVIL DA GOVERNADORA DO ESTADO\nEXTRA...
9,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO DO PRIMEIRO\r\nTERMO ADITIVO AO CONTRA...
10,19/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO DE LICITAÇÃO - TOMADA DE PREÇOS N° 015_2...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,AVISO DE LICITAÇÃO\nTOMADA DE PREÇOS N° 015/20...


In [48]:
print(materias_df.texto.iloc[50])

FUNDAÇÃO UNIVERSIDADE DO ESTADO DO RIO
GRANDE DO NORTE-FUERN
 
Portaria nº 2326/2019-GP/FUERN
 
Concede auxílio-doença e
licença para tratamento de saúde.
 
O Presidente da Fundação
Universidade do Estado do Rio Grande do Norte - FUERN, no uso de suas atribuições
legais e estatutárias, com fulcro no art. 63, §1°, da Lei Complementar nº 163,
de 05 de fevereiro de 1999, combinado com o art. 48 da Lei Complementar nº 308,
de 25 de outubro de 2005, e art.90 da Lei Complementar nº 122, de 30 de junho
de 1994;
CONSIDERANDO o
laudo da Junta Médica do Estado do RN, juntado ao processo nº 6290/2019 
UERN, de 15 de outubro de 2019;
CONSIDERANDO os termos do Despacho da DP/PROGEP, de 16 de outubro de 2019,
solicitando a feitura da portaria respectiva;
R E S O L V E:
Art. 1º. Conceder auxílio-doença e
respectiva licença para tratamento de saúde a(o) servidor(a) ELIZABETH DE
AZEVEDO SILVA BORGES, matrícula n.º 8873-0, ocupante do cargo de Técnico
Especializado  Técnico de Ambulatório, lotada no D

In [49]:
filename = "matérias_diário - {}".format(data_formatada.replace('/','_'))
materias_df.to_csv(('datasets/matérias/' + filename + '.csv'), index=False)

# Lendo datasets mais antigos

In [46]:
#diario = pd.read_csv('datasets\diários\diário - 28_09_2019.csv')
diario = pd.read_csv('datasets\diários\diário - 01_10_2019.csv')
diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...
1,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),NaN,NaN,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...
2,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),NaN,NaN,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


In [47]:
materias_diario = diario[diario['nome_materia'] != 'Não Possui'].copy()
materias_diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
7,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
8,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,NaN,NaN,NaN,SIN - ex-CV-018-2019-DETRAN-DER.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN),NaN,NaN,Extrato de Contrato,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [48]:
materias_diario['link_materia_oficial'] = materias_diario['materia'].apply(pegar_frame)
materias_diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial
7,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
8,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,NaN,NaN,NaN,SIN - ex-CV-018-2019-DETRAN-DER.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
10,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN),NaN,NaN,Extrato de Contrato,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...


In [49]:
materias_diario['texto'] = materias_diario['link_materia_oficial'].apply(pegar_texto)
materias_diario.head(3)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial,texto
7,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...
8,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,NaN,NaN,NaN,SIN - ex-CV-018-2019-DETRAN-DER.doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,*\r\nEXTRATO DO CONVÊNIO N° 018/2019 - SIN\n \...
10,1/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN),NaN,NaN,Extrato de Contrato,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,Secretaria de InfraEstrutura.\r\nDepartamento ...


In [50]:
filename = "matérias_diário - {}".format(materias_diario.iloc[0,0].replace('/','_'))
materias_diario.to_csv(('datasets/matérias/' + filename + '.csv'), index=False)

# Corrigindo datas

#### Diário

In [54]:
file_diario = pd.read_csv('datasets\diários\diário - 10_10_2019.csv') #.iloc[:,1:]
file_diario.head(1)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,10/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [52]:
file_diario.data_publicacao = '10/10/2019'
file_diario.to_csv('datasets\diários\diário - 10_10_2019.csv', index=False)

#### Matérias

In [10]:
file_materia = pd.read_csv('datasets\matérias\matérias_diário - 01_10_2019.csv') #.iloc[:,1:]
file_materia.head(1)

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial,texto
0,01/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...


In [8]:
file_materia.data_publicacao = '01/10/2019'
file_materia.to_csv('datasets\matérias\matérias_diário - 01_10_2019.csv', index=False)

# Corrigindo nome das colunas

In [5]:
import glob

In [6]:
diarios = glob.glob('datasets\diários\diário - *')
materias = glob.glob('datasets\matérias\matérias_diário - *')
print(diarios, materias)

['datasets\\diários\\diário - 01_10_2019.csv', 'datasets\\diários\\diário - 03_10_2019.csv', 'datasets\\diários\\diário - 08_10_2019.csv', 'datasets\\diários\\diário - 09_10_2019.csv', 'datasets\\diários\\diário - 10_10_2019.csv', 'datasets\\diários\\diário - 11_10_2019.csv', 'datasets\\diários\\diário - 16_10_2019.csv', 'datasets\\diários\\diário - 18_10_2019.csv', 'datasets\\diários\\diário - 28_09_2019.csv'] ['datasets\\matérias\\matérias_diário - 01_10_2019.csv', 'datasets\\matérias\\matérias_diário - 03_10_2019.csv', 'datasets\\matérias\\matérias_diário - 08_10_2019.csv', 'datasets\\matérias\\matérias_diário - 09_10_2019.csv', 'datasets\\matérias\\matérias_diário - 10_10_2019.csv', 'datasets\\matérias\\matérias_diário - 11_10_2019.csv', 'datasets\\matérias\\matérias_diário - 16_10_2019.csv', 'datasets\\matérias\\matérias_diário - 18_10_2019.csv', 'datasets\\matérias\\matérias_diário - 28_09_2019.csv']


#### Diários

In [12]:
new_cols = ['secao','subsecao1','subsecao2','subsecao3','subsecao4','subsecao5']

for d in diarios:
    diario = pd.read_csv(d)
    diario.rename(columns=dict(zip(diario.columns[2:8], new_cols)),inplace=True)
    diario.to_csv(d, index=False)
    display(diario.head(1))

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,01/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,03/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,08/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,NaN,Não Possui,Nenhuma matéria encontrada nessa data para ess...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,09/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,10/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia
0,28/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO DISPENSA DE LICITAÇÃO (TECPEL).doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


#### Matérias

In [13]:
for m in materias:
    materia = pd.read_csv(m)
    materia.rename(columns=dict(zip(materia.columns[2:8], new_cols)),inplace=True)
    materia.to_csv(m, index=False)
    display(materia.head(1))

,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,01/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,03/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO DO PRIMEIRO TERMO ADITIVO AO TERMO DE ...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,08/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,DEPARTAMENTO ESTADUAL DE IMPRENSA\nEXTRATO\r\n...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,09/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO DO II TERMO ADITIVO AO CONTRATO\r\r\nD...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,10/10/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),NaN,NaN,DEI - EXTRATO 1º TERMO DE ADITIVO (JMT - 3 MOT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,DEPARTAMENTO ESTADUAL DE IMPRENSA\nEXTRATO\r\n...


,data_publicacao,diario,secao,subsecao1,subsecao2,subsecao3,subsecao4,subsecao5,nome_materia,materia,link_materia_oficial,texto
0,28/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),NaN,NaN,NaN,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDE DISPENSA DE LICITAÇÃO\nProcesso...


# Teste

In [4]:
file = 'teste'
lista = [[1,2,3],[1,2,3],[1,2,3]]
df = pd.DataFrame(lista, columns=['a','b','c'])
df.to_csv('testando' + file + '.csv', index=False)

# O que fazer
##### Seleção de data
    Iterar por data (opcional)
##### Criar Funções (fazendo)
##### Minimizar gambiarras
##### Fazer navegação recursiva
##### Criar arquivo .py
##### Renomear Colunas DF
##### Exportar matérias txt
    Organizar pastas
##### Pegar texto minusculo sem tabela